In [ ]:
etherpad_url = "http://localhost:9001/"
test_pad_title = "New E2E Test Pad"
renamed_pad_title = "Renamed E2E Test Pad"
default_result_path = None
close_on_fail = False
transition_timeout = 10000

In [ ]:
import tempfile

work_dir = tempfile.mkdtemp()
if default_result_path is None:
    default_result_path = work_dir
work_dir

# ep_weave E2E Test - Single Pad

- Test data to prepare: Not required

In [ ]:
import importlib

import scripts.playwright
importlib.reload(scripts.playwright)

from scripts.playwright import *

await init_pw_context(close_on_fail=close_on_fail, last_path=default_result_path)

## Open etherpad_url and check if the search box is editable.


In [ ]:
index_page = None

async def _step(page):
    await page.goto(etherpad_url)

    await expect(page.locator(".hashview-search-box")).to_be_editable()

    global index_page
    index_page = page

await run_pw(_step)

## Fill the search box with test_pad_title and click the create button.

In [ ]:
import re

pad_page = None

async def _step(page):
    await page.locator(".hashview-search-box").fill(test_pad_title)
    await expect(page.locator('//button[text()="Create"]')).to_be_enabled()

    # Wait for a new window to open with a title starting with test_pad_title
    popup_future = page.wait_for_event('popup')
    await page.locator('//button[text()="Create"]').click()
    popup = await popup_future
    await expect(popup).to_have_title(re.compile(f"^{test_pad_title}"))

    await expect(popup.locator('//iframe[@name="ace_outer"]')).to_be_visible(timeout=transition_timeout)

    global pad_page
    pad_page = popup
    return popup

await run_pw(_step)

## Edit the pad by appending "Hello, World!" and verify that the text appears in the pad.

In [ ]:
async def extract_ace_inner_docbody(page):
    iframe_locator = page.frame_locator('iframe[name="ace_outer"]').frame_locator('iframe[name="ace_inner"]')
    await expect(iframe_locator.locator('#innerdocbody')).to_be_visible(timeout=transition_timeout)
    return iframe_locator.locator('#innerdocbody')

async def _step(page):
    inner_docbody = await extract_ace_inner_docbody(page)
    await expect(inner_docbody).to_be_editable(timeout=transition_timeout)
    inner_docbody.focus()

    # To move the cursor to the end of line, use the End key
    await inner_docbody.press("End")
    for _ in range(2):
        await inner_docbody.press("Enter")
    await inner_docbody.type("Hello, World!", delay=100)

await run_pw(_step)

## Refresh the index page and verify that the created pad is listed.

In [ ]:
async def _step(page):
    await index_page.reload()
    await expect(index_page.locator(f'text="{test_pad_title}"')).to_be_visible(timeout=transition_timeout)

    return index_page

await run_pw(_step)

## Search the created pad using the search box and verify that it only shows the created pad.

In [ ]:
async def _step(page):
    await page.locator(".hashview-search-box").fill(f'"{test_pad_title}"')
    await page.keyboard.press("Enter")
    await expect(page.locator('.hash-link')).to_have_count(1, timeout=transition_timeout)

await run_pw(_step)

## Rename the created pad

In [ ]:
async def _step(page):
    page = pad_page
    inner_docbody = await extract_ace_inner_docbody(page)
    await inner_docbody.focus()
    for _ in range(3):
        await inner_docbody.press("ArrowUp")
    await inner_docbody.press("End")
    for _ in range(len(test_pad_title) + 1):
        await inner_docbody.press("Backspace")
    await inner_docbody.type(renamed_pad_title, delay=100)
    await inner_docbody.press("Enter")

    await expect(page).to_have_title(re.compile(f"^{renamed_pad_title}"), timeout=transition_timeout)
    return page

await run_pw(_step)

## Refresh the index page and verify that the renamed pad is listed.

In [ ]:
import asyncio
import traceback

async def _step(page):
    # Reload the index page multiple times to ensure the change is reflected
    retries = 3
    for attempt in range(retries):
        try:
            await index_page.goto(etherpad_url)
            await expect(index_page.locator(f'text="{renamed_pad_title}"')).to_be_visible(timeout=1000)
            break
        except Exception as e:
            if attempt == retries - 1:
                raise
            print(f"Error occurred: {e}, retrying... ({attempt + 1}/{retries})")
            traceback.print_exc()
            await asyncio.sleep(1)  # Wait before retrying

    return index_page

await run_pw(_step)

## Search the created pad using the search box and verify that it only shows the created pad.

In [ ]:
async def _step(page):
    await page.locator(".hashview-search-box").fill(f'"{test_pad_title}"')
    await page.keyboard.press("Enter")
    await expect(page.locator('.hash-link')).to_have_count(0, timeout=transition_timeout)
    await page.locator(".hashview-search-box").fill(f'"{renamed_pad_title}"')
    await page.keyboard.press("Enter")
    await expect(page.locator('.hash-link')).to_have_count(1, timeout=transition_timeout)

await run_pw(_step)

Clean up

In [ ]:
await finish_pw_context()


In [ ]:
!rm -fr {work_dir}